### Exercise Sheet 2

#### 2.3 Solving an inverse Problem with TorchPhysics
We consider now the wave equation

\begin{align*}
    \partial_t^2 u &= c \, \partial_x^2 u, &&\text{ in } I_x \times I_t, \\
    u &= 0 , &&\text{ on } \partial I_x \times I_t, \\
    \partial_t u &= 0 , &&\text{ on } \partial I_x \times I_t, \\
    u(\cdot, 0) &= \sin(x) , &&\text{ in } I_x,
\end{align*}

with $I_x = [0, 2\pi]$ and $I_t = [0, 20]$. We are given a noisy dataset $\{(u_i, x_i, t_i)\}_{i=1}^N$ and aim to determine the corresponding value of $c$.

In [ ]:
!pip install torchphysics

In [1]:
import torchphysics as tp
import pytorch_lightning as pl
import torch
import math

# Here all parameters are defined:
t_min, t_max = 0.0, 20.0
width = 2 * math.pi 

# Number of training points 
N_pde = 20000

# Training parameters
train_iterations = 5000
learning_rate = 1.e-3

In [2]:
### TODO: Implement the spaces


### TODO: Define the domain omega and time interval 
I_x = ...
I_t = ...

### TODO: Create sampler for the PDE condition inside I_x x I_t
pde_sampler = ...

In [3]:
### TODO: Create the neural networks for the solution u and the learnable parameter c.
###       The model of u should contain 3 hidden layers with 50 neurons each and should have
###       X*T as an input space (order is important for the following cells).
###       For the parameter c use `tp.models.Parameter` and the initial value 1.0
model_u = ...
param_C = ...

In [4]:
### TODO: Define condition for the wave equation. Parameters can be passed to the condition
###       with the `parameter` keyword.
def pde_residual():
    pass

pde_condition = ...

In [ ]:
### Here, we load the data. First download it from GitHub and then read it with
### PyTorch. `in_data` contains combinations of X*T points and 'out_data' the 
### coressponding ampltidue of the wave.

!wget https://github.com/TomF98/torchphysics/raw/main/examples/workshop/FEMData/Data2_3/time_points.pt
!wget https://github.com/TomF98/torchphysics/raw/main/examples/workshop/FEMData/Data2_3/space_coords.pt
!wget https://github.com/TomF98/torchphysics/raw/main/examples/workshop/FEMData/Data2_3/wave_data.pt

In [ ]:
fe_time = torch.load("time_points.pt")
fe_space = torch.load("space_coords.pt")
out_data = torch.load("wave_data.pt")

in_data = torch.zeros((len(fe_time), len(fe_space), 2))
in_data[:, :, :1] = fe_space
in_data[:, :, 1] = fe_time

in_data = in_data.reshape(-1, 2)

print("Data has the shape:")
print(in_data.shape, out_data.shape)

In [6]:
### TODO: Randomly shuffle the data from the previous cell, add 1% of articfical noise to the `out_data`
###       and then select for the training only the first half of the data batch.
###       Hint: for the random shuffle `torch.randperm` is useful and for constructing noise 
###       use: `0.01 * torch.randn_like(out_data) * out_data`



In [7]:
### TODO: Transform the data from the previous cell into `tp.spaces.Points` objects, to
###       assign them a space and enable TorchPhysics to work with them:
in_data_points = ...
out_data_points = ...

### Here we create a DataLoader, that passes the above data to the conditions and
### also controls the batch size, the device (CPU or GPU) and more...
### And also the condition, that fits the given model to the data
data_loader = tp.utils.PointsDataLoader((in_data_points, out_data_points), batch_size=len(in_data))

In [8]:
### Data condition, that fits the model to the given data:
data_condition = tp.conditions.DataCondition(module=model_u,
                                             dataloader=data_loader,
                                             norm=2, use_full_dataset=True,
                                             weight=100) 

In [ ]:
### Start training with Adam:
optim = tp.OptimizerSetting(optimizer_class=torch.optim.Adam, lr=learning_rate)
solver = tp.solver.Solver(train_conditions=[data_condition, pde_condition], optimizer_setting=optim)


trainer = pl.Trainer(gpus=1,
                     max_steps=train_iterations,
                     logger=False,
                     benchmark=True)
                     
trainer.fit(solver)

In [ ]:
### For better results in the inverse problem, switching to LBFGS is useful:
optim = tp.OptimizerSetting(optimizer_class=torch.optim.LBFGS, lr=0.5, optimizer_args={'max_iter': 2})
pde_condition.sampler = pde_condition.sampler.make_static()
solver = tp.solver.Solver([pde_condition, data_condition], optimizer_setting=optim)

trainer = pl.Trainer(devices=1, accelerator="gpu",
                     num_sanity_val_steps=0,
                     benchmark=True,
                     max_steps=2500, 
                     logger=False, 
                     enable_checkpointing=False)
trainer.fit(solver)

In [ ]:
print("Correct value of c is: 0.742")
print("With PINNs we computed the value:", param_C.as_tensor.item())
print("Relative difference is:", abs(0.742 - param_C.as_tensor.item()) / 0.742)

In [ ]:
### We can also plot the solution that we learned
plot_domain = tp.domains.Parallelogram(X*T, [0, 0], [width, 0], [0, t_max])
plot_sampler = tp.samplers.PlotSampler(plot_domain, 1000)
fig = tp.utils.plot(model_u, lambda u: u, plot_sampler, plot_type="contour_surface")

In [ ]:
# Or an animation:
anim_sampler = tp.samplers.AnimationSampler(I_x, I_t, 200, n_points=250)
fig, anim = tp.utils.animate(model_u, lambda u: u, anim_sampler, ani_speed=40)
anim.save('wave-eq.gif')
# On Google colab you have at the left side a tab with a folder. There you should find the gif and can watch it.